# Prototype: an efficient, approximate running maximum filter

The window size (units of time) is denoted as $p$.

Constraints:
* The filter uses a prescribed number of bins, $n$, to store the history.
  Typically $n$ << the number of time steps the model would take to cover the window.
  For instance, a 1-year running mean may use just 1 bin. In that case, the maximum during very first year will be set to "missing value".
  In the entire subsequent year, the maximum will be equal to the maximum over the first year. And so on.
  By definition, this means the running maximum will be an approximation that ignores all recent history (of duration window size / # bins).
* We have no advance knowledge about the model time step $\Delta t$, which may be variable.

In [ ]:
import numpy as np
from matplotlib import pyplot

p = 1.0                           # window size time units
delta_t = 1.0 / 24.0 / 6.0 / 365  # model time step in time units
duration = 5.0                    # simulation duration in time units
n = 1                             # number of bins for history (covering period p)
missing_value = -2.0              # value to return while the simulation has not covered 1 window size yet

In [ ]:
class Filter:
    def __init__(self):
        self.history = np.zeros((n,))
        self.previous_time = 0.0
        self.previous_value = 0
        self.ibin = -1
        self.bin_end_time = 0.0
        self.max = missing_value
        self.complete = False

    def __call__(self, now: float, value: float) -> float:
        if self.ibin == -1:
            self.bin_end_time = now + p / n
            self.ibin = 0

        while now >= self.bin_end_time:
            # Interpolate to value at right bin time
            w = (self.bin_end_time - self.previous_time) / (now - self.previous_time)
            self.previous_time = self.bin_end_time
            self.previous_value += w * (value - self.previous_value)

            # Increment the bin we are completing (history[ibin]) and mean
            # bin_end_value = (1 - w) * self.previous_value + w * value
            # self.history[self.ibin] += 0.5 * dt * (self.previous_value + bin_end_value) / p
            self.history[self.ibin] = np.maximum(
                self.history[self.ibin], self.previous_value
            )
            self.complete = self.complete or self.ibin == n - 1
            if self.complete:
                self.max = np.max(self.history[:, ...], axis=0)
            self.ibin = 0 if self.ibin == n - 1 else self.ibin + 1
            self.history[self.ibin] = self.previous_value

            self.bin_end_time += p / n

        # Update the maximum of the current bin
        self.history[self.ibin] = np.maximum(self.history[self.ibin], value)

        # Store current time and value to enable linear interpolation in subsequent call.
        self.previous_time = now
        self.previous_value = value
        return self.max

In [ ]:
# Calculate and plot variable for which to compute the running maximum
times = np.arange(0, duration, delta_t)
values = np.sin(2 * np.pi * times / 3)

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(times, values)
ax.grid()

In [ ]:
# Compute and plot the running maximum
filter = Filter()
filtered = np.empty_like(values)
for i, (time, value) in enumerate(zip(times, values)):
    filtered[i] = filter(time, value)

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(times, values)
ax.plot(times, filtered)
ax.grid()

In [ ]:
# Compare running maximum with analytical solution
# (this requires the window size to be a multiple of the model time step)
assert (
    abs(p % delta_t) < 1e-15
), "Window size %s is not a multiple of the model time step %s. Residual: %s" % (
    p,
    delta_t,
    p % delta_t,
)
nstep = int(round(p / delta_t))
analytical = np.full_like(values, missing_value)
for i, (time, value) in enumerate(zip(times, values)):
    if i >= nstep:
        analytical[i] = values[i - nstep : i + 1].max()

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(times, filtered - analytical)
ax.grid()

In [ ]:
# Now try with randomly varying time step
filter = Filter()
time = 0.0
rtimes, rvalues, rfiltered = [], [], []
while time < duration:
    value = np.sin(2 * np.pi * time / 3)
    rtimes.append(time)
    rvalues.append(value)
    rfiltered.append(filter(time, value))
    dt = 2 * delta_t * np.random.rand()
    time += dt

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(rtimes, rvalues)
ax.plot(rtimes, rfiltered)
ax.grid()

In [ ]:
# Now try the running maximum filter in FABM itself
import pyfabm

config = {
    "instances": {
        "max": {
            "model": "surface_temporal_maximum",
            "parameters": {"window": p, "n": n, "missing_value": missing_value},
        }
    }
}
m = pyfabm.Model(config)
invar = m.dependencies["max/source"]
outvar = m.diagnostic_variables["max/max"]
invar.value = missing_value
m.cell_thickness = 1.0
m.start()
fabm_filtered = np.empty_like(values)
for i, (time, value) in enumerate(zip(times, values)):
    invar.value = value
    m.get_sources(time)
    fabm_filtered[i] = outvar.value

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(times, values)
ax.plot(times, fabm_filtered)
ax.grid()

fig, ax = pyplot.subplots(figsize=(15, 4))
ax.plot(times, fabm_filtered - analytical)
ax.grid()

In [ ]:
fig, ax = pyplot.subplots()
ax.plot(filtered, fabm_filtered, ".")
ax.grid()